# Cross-Asset Transfer (PPO): train on BTC → test on ETH/SOL

**Idea:** train the same PPO policy on episodes retrieved for one asset (e.g. BTCUSDT) and evaluate it on held-out episodes of other assets (ETHUSDT, SOLUSDT).

**Artifact:** transfer matrix `(train asset × test asset)` with out-of-sample performance.

Why this is a “WOW” signal:
- If an agent trained on one market retains edge on another, it suggests the policy exploits **structural** patterns (regimes / shapes), not just asset-specific quirks.


## Setup
Deps: `pandas`, `numpy`, `plotly`, plus RL deps `gymnasium`, `stable-baselines3`.

Install (if needed):
```bash
cd python-sdk
/Users/serg/projects/prod/ai_patterns/.venv/bin/python -m pip install -e .
/Users/serg/projects/prod/ai_patterns/.venv/bin/python -m pip install pandas numpy plotly gymnasium stable-baselines3
```


In [1]:
import os
import json
import gzip
import time
from dataclasses import dataclass
from pathlib import Path
from datetime import datetime, timezone

import numpy as np
import pandas as pd
import plotly.express as px

from aipricepatterns import Client

try:
    import gymnasium as gym
    from gymnasium import spaces
except Exception as e:
    raise ImportError("gymnasium is required. Install: pip install gymnasium") from e

try:
    from stable_baselines3 import PPO
    from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
except Exception as e:
    raise ImportError("stable-baselines3 is required. Install: pip install stable-baselines3") from e

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)


## Parameters
This can be heavy if you use many anchors across many assets. The defaults are tuned for “notebook speed”; bump them for final runs.


In [2]:
BASE_URL = os.getenv("AIPP_BASE_URL", "https://aipricepatterns.com/api/rust")
API_KEY = os.getenv("AIPP_API_KEY")

# Comma-separated list
ASSETS = [s.strip() for s in os.getenv("AIPP_ASSETS", "BTCUSDT,ETHUSDT,SOLUSDT").split(",") if s.strip()]
INTERVAL = os.getenv("AIPP_RL_INTERVAL", "1h")

ANCHOR_POINTS = int(os.getenv("AIPP_SWEEP_ANCHORS", "200"))
LOOKBACK_DAYS = int(os.getenv("AIPP_SWEEP_LOOKBACK_DAYS", "120"))
FORECAST_HORIZON = int(os.getenv("AIPP_RL_HORIZON", "24"))
EPISODES_PER_ANCHOR = int(os.getenv("AIPP_SWEEP_EPISODES_PER_ANCHOR", "10"))
MIN_SIMILARITY = float(os.getenv("AIPP_RL_MIN_SIMILARITY", "0.70"))
SAMPLING_STRATEGY = os.getenv("AIPP_RL_SAMPLING_STRATEGY", "uniform")

# Feature set for observations
OBS_SET = os.getenv("AIPP_OBS_SET", "price_log_vol_dd_cumret_similarity")

# Optional friction
FEE_PCT = float(os.getenv("AIPP_FEE_PCT", "0.00"))
SLIP_PCT = float(os.getenv("AIPP_SLIP_PCT", "0.00"))
ROUND_TRIP = bool(int(os.getenv("AIPP_ROUND_TRIP", "0")))

# PPO budget
TRAIN_TIMESTEPS = int(os.getenv("AIPP_PPO_TIMESTEPS", "15000"))
N_ENVS = int(os.getenv("AIPP_PPO_ENVS", "8"))
SEED = int(os.getenv("AIPP_SEED", "7"))

# Caps to keep runtime/memory sane
MAX_TRAIN_EPISODES = int(os.getenv("AIPP_MAX_TRAIN_EPISODES", "4000"))
MAX_TEST_EPISODES = int(os.getenv("AIPP_MAX_TEST_EPISODES", "1200"))

# Include a MULTI row: train on all assets pooled (0/1)
INCLUDE_MULTI_TRAIN = bool(int(os.getenv("AIPP_MULTI_ASSET_TRAIN", "0")))

NOTEBOOK_DIR = Path.cwd()
CACHE_DIR = Path(os.getenv("AIPP_RESEARCH_CACHE_DIR", str(NOTEBOOK_DIR / "_cache")))
CACHE_DIR.mkdir(parents=True, exist_ok=True)

print("Base URL:", BASE_URL)
print("Assets:", ASSETS, "interval:", INTERVAL)
print(f"anchors={ANCHOR_POINTS} lookbackDays={LOOKBACK_DAYS} horizon={FORECAST_HORIZON}")
print(f"episodes/anchor={EPISODES_PER_ANCHOR} minSim={MIN_SIMILARITY} strategy={SAMPLING_STRATEGY}")
print(f"OBS_SET={OBS_SET}")
print(f"feePct={FEE_PCT} slipPct={SLIP_PCT} roundTrip={ROUND_TRIP}")
print(f"TRAIN_TIMESTEPS={TRAIN_TIMESTEPS} N_ENVS={N_ENVS} seed={SEED}")
print(f"MAX_TRAIN_EPISODES={MAX_TRAIN_EPISODES} MAX_TEST_EPISODES={MAX_TEST_EPISODES}")


Base URL: https://aipricepatterns.com/api/rust
Assets: ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'] interval: 1h
anchors=200 lookbackDays=120 horizon=24
episodes/anchor=10 minSim=0.7 strategy=uniform
OBS_SET=price_log_vol_dd_cumret_similarity
feePct=0.0 slipPct=0.0 roundTrip=False
TRAIN_TIMESTEPS=15000 N_ENVS=8 seed=7
MAX_TRAIN_EPISODES=4000 MAX_TEST_EPISODES=1200


## Fetch & cache episodes per asset
We cache raw episodes (with transitions) per asset as `.json.gz` to avoid repeated API calls.


In [3]:
def _safe_float(x, default=np.nan):
    try:
        return float(x)
    except Exception:
        return float(default)

def _map_suggested_action_to_pos(x) -> int:
    if x is None:
        return 0
    if isinstance(x, (int, float)):
        v = int(x)
        if v in (-1, 0, 1):
            return v
        if v in (0, 1, 2):
            return 1 if v == 1 else (-1 if v == 2 else 0)
        return 0
    s = str(x).strip().lower()
    if s in ("hold","flat","none","neutral","wait"): return 0
    if s in ("long","buy","bull","up"): return 1
    if s in ("short","sell","bear","down"): return -1
    return 0

def cache_path_for_asset(asset: str) -> Path:
    safe = asset.replace("/", "_")
    return CACHE_DIR / f"09_transfer_eps_{safe}_{INTERVAL}_{ANCHOR_POINTS}.json.gz"

def load_or_fetch_asset(asset: str) -> list[dict]:
    p = cache_path_for_asset(asset)
    if p.exists():
        with gzip.open(p, "rt", encoding="utf-8") as f:
            data = json.load(f)
        print(asset, "loaded", len(data), "episodes from", str(p))
        return data

    client = Client(base_url=BASE_URL, api_key=API_KEY)
    now_ms = int(time.time() * 1000)
    start_ms = now_ms - LOOKBACK_DAYS * 24 * 60 * 60 * 1000
    anchors = np.linspace(start_ms, now_ms, num=ANCHOR_POINTS, dtype=np.int64).tolist()
    out = []
    for i, anchor_ts in enumerate(anchors, start=1):
        res = client.get_rl_episodes(
            symbol=asset,
            interval=INTERVAL,
            anchor_ts=int(anchor_ts),
            forecast_horizon=FORECAST_HORIZON,
            num_episodes=EPISODES_PER_ANCHOR,
            min_similarity=MIN_SIMILARITY,
            include_actions=True,
            reward_type="returns",
            sampling_strategy=SAMPLING_STRATEGY,
        )
        eps = res.get("episodes") if isinstance(res, dict) else None
        if isinstance(eps, list):
            for ep in eps:
                ts = ep.get("transitions")
                if not isinstance(ts, list) or len(ts) < 2:
                    continue
                out.append({
                    "asset": asset,
                    "anchorTs": int(anchor_ts),
                    "similarity": _safe_float(ep.get("similarity"), np.nan),
                    "transitions": ts,
                })
        if i % 25 == 0:
            print(asset, f"{i}/{len(anchors)} anchors, episodes={len(out)}")
        time.sleep(0.02)

    out = [e for e in out if np.isfinite(e.get("similarity", np.nan))]
    out.sort(key=lambda e: e["anchorTs"])
    with gzip.open(p, "wt", encoding="utf-8") as f:
        json.dump(out, f)
    print(asset, "wrote", len(out), "episodes to", str(p))
    return out

asset_eps = {a: load_or_fetch_asset(a) for a in ASSETS}
{a: len(v) for a,v in asset_eps.items()}


BTCUSDT 25/200 anchors, episodes=250
BTCUSDT 50/200 anchors, episodes=500
BTCUSDT 75/200 anchors, episodes=750
BTCUSDT 100/200 anchors, episodes=1000
BTCUSDT 125/200 anchors, episodes=1250
BTCUSDT 150/200 anchors, episodes=1500
BTCUSDT 175/200 anchors, episodes=1750
BTCUSDT 200/200 anchors, episodes=1996
BTCUSDT wrote 1996 episodes to /Users/serg/projects/prod/ai_patterns/python-sdk/research/_cache/09_transfer_eps_BTCUSDT_1h_200.json.gz
ETHUSDT 25/200 anchors, episodes=250
ETHUSDT 50/200 anchors, episodes=500
ETHUSDT 75/200 anchors, episodes=750
ETHUSDT 100/200 anchors, episodes=1000
ETHUSDT 125/200 anchors, episodes=1250
ETHUSDT 150/200 anchors, episodes=1500
ETHUSDT 175/200 anchors, episodes=1750
ETHUSDT 200/200 anchors, episodes=1997
ETHUSDT wrote 1997 episodes to /Users/serg/projects/prod/ai_patterns/python-sdk/research/_cache/09_transfer_eps_ETHUSDT_1h_200.json.gz
SOLUSDT 25/200 anchors, episodes=250
SOLUSDT 50/200 anchors, episodes=500
SOLUSDT 75/200 anchors, episodes=750
SOLUSDT

{'BTCUSDT': 1996, 'ETHUSDT': 1997, 'SOLUSDT': 1997}

## Train/Test split per asset (time-based)
We split by `anchorTs` so the test set is chronologically after train.


In [4]:
def split_train_test(eps: list[dict], train_frac: float = 0.8):
    eps = sorted(eps, key=lambda e: e["anchorTs"])
    cut = int(train_frac * len(eps))
    train = eps[:cut]
    test = eps[cut:]
    if len(train) > MAX_TRAIN_EPISODES:
        train = train[-MAX_TRAIN_EPISODES:]
    if len(test) > MAX_TEST_EPISODES:
        test = test[:MAX_TEST_EPISODES]
    return train, test

splits = {}
for a, eps in asset_eps.items():
    tr, te = split_train_test(eps, train_frac=0.8)
    splits[a] = (tr, te)
    if eps:
        tr0 = datetime.fromtimestamp(tr[0]["anchorTs"]/1000, tz=timezone.utc) if tr else None
        tr1 = datetime.fromtimestamp(tr[-1]["anchorTs"]/1000, tz=timezone.utc) if tr else None
        te0 = datetime.fromtimestamp(te[0]["anchorTs"]/1000, tz=timezone.utc) if te else None
        te1 = datetime.fromtimestamp(te[-1]["anchorTs"]/1000, tz=timezone.utc) if te else None
        print(a, "train", len(tr), tr0, "→", tr1, "| test", len(te), te0, "→", te1)

None


BTCUSDT train 1596 2025-08-21 18:01:04.079000+00:00 → 2025-11-25 15:07:23.978000+00:00 | test 400 2025-11-25 15:07:23.978000+00:00 → 2025-12-19 18:01:04.079000+00:00
ETHUSDT train 1597 2025-08-21 18:01:29.855000+00:00 → 2025-11-25 15:07:49.754000+00:00 | test 400 2025-11-25 15:07:49.754000+00:00 → 2025-12-19 18:01:29.855000+00:00
SOLUSDT train 1597 2025-08-21 18:01:54.193000+00:00 → 2025-11-25 15:08:14.092000+00:00 | test 400 2025-11-25 15:08:14.092000+00:00 → 2025-12-19 18:01:54.193000+00:00


## Environment (same as 08, but reused for transfer)
We build a small episodic environment on returns with optional friction.


In [5]:
@dataclass
class ObsConfig:
    name: str
    features: list[str]

OBS_MAP = {
    "ret_only": ["ret"],
    "price_log": ["price", "log_price"],
    "price_log_vol": ["price", "log_price", "vol"],
    "price_log_vol_dd_cumret": ["price", "log_price", "vol", "dd", "cumret"],
    "price_log_vol_dd_cumret_similarity": ["price", "log_price", "vol", "dd", "cumret", "similarity"],
    "all": ["price", "log_price", "vol", "dd", "cumret", "similarity", "suggested_pos"],
    "suggested_only": ["suggested_pos"],
}
OBS_FEATURES = OBS_MAP.get(OBS_SET)
if OBS_FEATURES is None:
    raise ValueError(f"Unknown OBS_SET={OBS_SET}. Options: {sorted(OBS_MAP.keys())}")
print("Using obs features:", OBS_FEATURES)

def build_episode_arrays(ep: dict):
    ts = ep["transitions"]
    rets = []
    sugg = []
    for t in ts:
        if not isinstance(t, dict):
            continue
        rets.append(_safe_float(t.get("ret", t.get("return", 0.0)), 0.0))
        sugg.append(_map_suggested_action_to_pos(t.get("suggestedAction")))
    rets = np.asarray(rets, dtype=np.float32)
    sugg = np.asarray(sugg, dtype=np.int8)
    n = len(rets)
    price = np.ones(n, dtype=np.float32)
    for i in range(1, n):
        price[i] = max(1e-6, price[i-1] * (1.0 + rets[i-1]))
    log_price = np.log(price).astype(np.float32)
    cumret = np.cumsum(rets).astype(np.float32)
    peak = np.maximum.accumulate(cumret)
    dd = (cumret - peak).astype(np.float32)
    w = 10
    vol = np.zeros(n, dtype=np.float32)
    for i in range(n):
        a = max(0, i - w + 1)
        vol[i] = float(np.std(rets[a:i+1]))
    sim = float(ep.get("similarity", 0.0))
    similarity = np.full(n, sim, dtype=np.float32)
    return {
        "ret": rets,
        "price": price,
        "log_price": log_price,
        "vol": vol,
        "dd": dd,
        "cumret": cumret,
        "similarity": similarity,
        "suggested_pos": sugg.astype(np.float32),
    }

class EpisodeEnv(gym.Env):
    metadata = {"render_modes": []}
    def __init__(self, episodes: list[dict], obs_features: list[str], fee_pct: float, slip_pct: float, round_trip: bool, seed: int = 0):
        super().__init__()
        self.episodes = episodes
        self.obs_features = list(obs_features)
        self.fee = float(fee_pct) / 100.0
        self.slip = float(slip_pct) / 100.0
        self.round_trip = bool(round_trip)
        self.rng = np.random.default_rng(seed)
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(self.obs_features),), dtype=np.float32)
        self._t = 0
        self._pos = 0
        self._arr = None

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        ep = self.episodes[int(self.rng.integers(0, len(self.episodes)))]
        self._arr = build_episode_arrays(ep)
        self._t = 0
        self._pos = 0
        return self._obs(), {}

    def _obs(self):
        return np.asarray([float(self._arr[k][self._t]) for k in self.obs_features], dtype=np.float32)

    def step(self, action):
        a = int(action)
        self._pos = 0 if a == 0 else (1 if a == 1 else -1)
        r = float(self._arr["ret"][self._t])
        pnl = float(self._pos) * r
        cost = (self.fee + self.slip)
        if self.round_trip:
            cost *= 2.0
        pnl -= abs(self._pos) * cost
        self._t += 1
        terminated = self._t >= (len(self._arr["ret"]) - 1)
        obs = self._obs() if not terminated else np.zeros((len(self.obs_features),), dtype=np.float32)
        return obs, float(pnl), terminated, False, {}


Using obs features: ['price', 'log_price', 'vol', 'dd', 'cumret', 'similarity']


## Train on one asset, test on another
We train PPO with VecNormalize on the train asset episodes, then evaluate on each test asset with the **train normalization stats**.


In [6]:
def train_ppo(train_episodes: list[dict], obs_features: list[str]):
    def make_env(seed_offset=0):
        return EpisodeEnv(train_episodes, obs_features, fee_pct=FEE_PCT, slip_pct=SLIP_PCT, round_trip=ROUND_TRIP, seed=SEED + seed_offset)
    vec = DummyVecEnv([lambda i=i: make_env(i) for i in range(N_ENVS)])
    vec = VecNormalize(vec, norm_obs=True, norm_reward=False, clip_obs=10.0)
    model = PPO("MlpPolicy", vec, verbose=0, seed=SEED, n_steps=256, batch_size=256)
    model.learn(total_timesteps=TRAIN_TIMESTEPS)
    return model, vec.obs_rms

def eval_ppo(model, obs_rms, test_episodes: list[dict], obs_features: list[str], n_eval_eps: int = 250) -> dict:
    rng = np.random.default_rng(SEED + 999)
    n_eval = min(n_eval_eps, len(test_episodes))
    picks = rng.choice(len(test_episodes), size=n_eval, replace=False)
    pnls = []
    for idx in picks:
        ep = test_episodes[int(idx)]
        arr = build_episode_arrays(ep)
        pnl = 0.0
        pos = 0
        for t in range(len(arr["ret"]) - 1):
            obs = np.asarray([float(arr[k][t]) for k in obs_features], dtype=np.float32)
            # normalize obs using train obs_rms
            o = obs.reshape((1, -1))
            if obs_rms is not None:
                o = (o - obs_rms.mean) / np.sqrt(obs_rms.var + 1e-8)
                o = np.clip(o, -10.0, 10.0)
            action, _ = model.predict(o, deterministic=True)
            a = int(action[0])
            pos = 0 if a == 0 else (1 if a == 1 else -1)
            r = float(arr["ret"][t])
            pnl += float(pos) * r
            cost = (float(FEE_PCT) + float(SLIP_PCT)) / 100.0
            if ROUND_TRIP:
                cost *= 2.0
            pnl -= abs(pos) * cost
        pnls.append(float(pnl))
    pnls = np.asarray(pnls, dtype=float)
    return {
        "n": int(len(pnls)),
        "avgPnL": float(np.mean(pnls)) if len(pnls) else float("nan"),
        "winrate": float(np.mean(pnls > 0)) if len(pnls) else float("nan"),
        "p05": float(np.quantile(pnls, 0.05)) if len(pnls) else float("nan"),
        "p95": float(np.quantile(pnls, 0.95)) if len(pnls) else float("nan"),
    }


In [7]:
rows = []
for train_asset in ASSETS:
    train_eps, _ = splits[train_asset]
    if len(train_eps) < 50:
        print("skip train", train_asset, "too few episodes", len(train_eps))
        continue
    print("\nTraining on", train_asset, "episodes", len(train_eps))
    model, obs_rms = train_ppo(train_eps, OBS_FEATURES)
    for test_asset in ASSETS:
        _, test_eps = splits[test_asset]
        m = eval_ppo(model, obs_rms, test_eps, OBS_FEATURES, n_eval_eps=min(250, len(test_eps)))
        rows.append({
            "trainAsset": train_asset,
            "testAsset": test_asset,
            **m,
        })

if INCLUDE_MULTI_TRAIN and len(ASSETS) > 1:
    pooled = []
    for a in ASSETS:
        pooled.extend(splits[a][0])
    if len(pooled) > MAX_TRAIN_EPISODES:
        pooled = pooled[-MAX_TRAIN_EPISODES:]
    print("\nTraining MULTI on pooled episodes", len(pooled))
    model, obs_rms = train_ppo(pooled, OBS_FEATURES)
    for test_asset in ASSETS:
        _, test_eps = splits[test_asset]
        m = eval_ppo(model, obs_rms, test_eps, OBS_FEATURES, n_eval_eps=min(250, len(test_eps)))
        rows.append({
            "trainAsset": "MULTI",
            "testAsset": test_asset,
            **m,
        })

transfer = pd.DataFrame(rows)
transfer.sort_values(["trainAsset","testAsset"]).reset_index(drop=True)



Training on BTCUSDT episodes 1596

Training on ETHUSDT episodes 1597

Training on SOLUSDT episodes 1597


,trainAsset,testAsset,n,avgPnL,winrate,p05,p95
0,BTCUSDT,BTCUSDT,250,5.696200,1.000,1.587805,11.812770
1,BTCUSDT,ETHUSDT,250,6.034798,0.996,1.657480,11.645300
2,BTCUSDT,SOLUSDT,250,10.148122,0.996,3.125480,24.164260
3,ETHUSDT,BTCUSDT,250,5.080748,1.000,1.156200,11.146580
4,ETHUSDT,ETHUSDT,250,5.624513,0.996,1.180700,11.039870
5,ETHUSDT,SOLUSDT,250,9.724921,0.996,2.745465,23.995920
6,SOLUSDT,BTCUSDT,250,5.473426,0.976,0.710270,11.286230
7,SOLUSDT,ETHUSDT,250,6.064503,0.996,1.250065,11.591165
8,SOLUSDT,SOLUSDT,250,10.245008,0.988,2.832300,23.400655


## Transfer matrix
Primary view: heatmap of `avgPnL` for (train × test).


In [8]:
mat = transfer.pivot(index="trainAsset", columns="testAsset", values="avgPnL")
mat


testAsset,BTCUSDT,ETHUSDT,SOLUSDT
trainAsset,,,
BTCUSDT,5.696200,6.034798,10.148122
ETHUSDT,5.080748,5.624513,9.724921
SOLUSDT,5.473426,6.064503,10.245008


In [9]:
fig = px.imshow(
    mat,
    aspect="auto",
    origin="lower",
    title=f"Cross-asset transfer: avgPnL (OBS_SET={OBS_SET}, timesteps={TRAIN_TIMESTEPS})",
    labels={"x": "testAsset", "y": "trainAsset", "color": "avgPnL"},
)
fig.update_layout(height=520)
fig


## How to interpret
- Strong diagonal + weak off-diagonals: mostly asset-specific learning.
- Meaningful off-diagonals: evidence of transferable policy structure.
- If `MULTI` row (optional) dominates: multi-asset training improves robustness.
